#TXINE. Práctica2: Web Scraping
### Adolfo Núñez Fernández, Outono 2015 
## Requisitos

Extraer da web de referencia http://www.invertia.com/mercados/bolsa/indices/ibex-35/acciones-ib011ibex35 as cotizacións publicadas das empresas do IBEX35. O programa terá dous modos de operación:
* Escritura por pantalla ou a ficheiro de todas as cotizacións extraídas e os seus datos asociados.
* Monitorización cada X segundos dun determinado valor bursatil a partires do identificador proporcionado polo usuario.

## Explicación do código

O programa estructúrase a partires dun **main** que ofrece un menú de opcións ao usuario e funcións que realizan as operacións solicitadas. A continuación amósase a súa organización e funcionamiento. 

### Imports



In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import time

As librarías empregadas son:

**BeautifulSoup:** Para extraer datos de ficheiros HTML e XML.

**pandas:** Para traballar con estruturas de datos flexibles.

**requests:** Permite obter o código dunha páxina mediante unha petición.

**time:** Libraría estandar de python para traballar coa data actual do sistema e facer pausas na execución.



### Main e función menu()

A función **main** permite a interacción do usuario co programa a través dun menú de catro opcións:
* [1] Escritura das cotizacións na pantalla.
* [2] Gardado das cotizacións a ficheiro **.csv**.  
* [3] Monitorización dun valor bursátil introducido polo usuario consultado cada X segundos, sendo X un valor introducido polo usuario.
* [4] Saída do programa.

O programa principal presenta o menú mentres non se escolle algunha das opcións válidas. 

Para a opción 1, o programa chama á función **getBolsa()** e imprime por pantalla o resultado.

Na segunda opción, o programa obtén igualmente os resultados mediante a función **getBolsa()**, e despois solicítalle ao usuario que proporcione o nome do ficheiro onde quere almacenar os datos. Se o usuario non proporciona ningún, automáticamente créase un por defecto (bolsa_YYmmdd_HHMMSS, sendo YY, mm, dd, HH, MM e SS ano, mes, día, hora, minutos e segundos respectivamente). Finalmente realízase o gardado do dataframe no ficheiro, notificando ao usuario oa fin da operación por pantalla.

A terceira opción solicítalle ao usuario o nome do valor bursátil que desexa consultar e máis o intervalo de tempo en segundos que empregará para obter os datos (pedirá o valor ata que o usuario introduza un número superior a 5 segundos). Antes de empezar a obter os valores coa periodicidade indicada, o usuario é notificado de que para saír do proceso de amosado de datos deberá premer as teclas **Ctrl+c**, voltando deste xeito de novo ao menú principal da aplicación. Cos datos de valor bursátil e período de tempo, o programa consulta con dita periodicidade os datos da url, buscando mediante a función ** getIdentificadorBursatil(valor)** os correspondentes ao indicador solicitado. Os resultados amósanse consecutivamente por pantalla en caso de atoparse o índice. En caso contrario, notifícaselle a súa ausencia ao usuario, tamén por pantalla.

A opción 4 remata a execución do programa, presentando unha mensaxe de despedida por pantalla. 

In [ ]:
def menu():
    print("\nEscolla unha opción:\n\
    [1] Cotización por pantalla\n\
    [2] Cotización a fichero\n\
    [3] Cotización dun valor concreto\n\
    [4] Saír")

if __name__ == "__main__":
    ''' Programa principal. Chamada ao menú de opcións '''

    # Opción inicial para que entre no bucle
    opcion = 0
    
    while opcion != 4:
        if opcion == 1: # escritura por pantalla
            df = getBolsa()
            print(df)
            
        elif opcion == 2: # escritura a ficheiro
            df = getBolsa()            

            # O usuario pode seleccionar o nome do ficheiro CSV
            fname = input('Nome do ficheiro: ')
            if len(fname) == 0:
                fname = "bolsa_{0}.csv".format(time.strftime("%Y%m%d_%H%M%S"))
            else:
                fname += '.csv'
            df.to_csv(fname)
                
            print('Táboa de valores gardada correctamente no ficheiro '\
            '"{0}"'.format(fname))

        elif opcion == 3: # Consulta cada X segs
            valor = input('Identificador do valor '\
            'bursatil que desexa consultar:')
            
            segundos = 0

            # O usuario é forzado a facer unha consulta cun retardo mínimo
            while segundos < 5:
                segundos = int(input('Periodicidade (s) [mínimo 5 segundos]:'))

            print('Prema "Ctrl+c" se desexa voltar ao menú')
            
            # Se existe o índice, devolvemos so seus valores cada X segundos
            try:
                while getIdentificadorBursatil(valor):            
                    time.sleep(segundos)
            except KeyboardInterrupt:
                pass
        else:
            pass
     
        #chamada ao menú
        menu()
        # Recóllese a opción do usuario
        opcion = int(input('Opción: '))
        
    if opcion == 4:
        print("Adeus")


### Función getBolsa()

Obtén as cotizacións de bolsa a partires da url de referencia. Coa libraría **requests** (http://docs.python-requests.org/en/latest/) obtense o texto da páxina, e mediante o uso da libraría **BeautifulSoup** (http://www.crummy.com/software/BeautifulSoup/bs4/doc/#) fanse búscas no HTML obtido.

O comportamento da función é o seguinte: recuperar a información desexada eliminando columnas con datos gráficos e limpando o texto. Para iso o que se fai é obter a táboa do HTML mediante **BeautifulSoup**. A partires de ahí faise unha busca das filas (etiquetas **tr**) e para cada unha delas faise unha busca das celas (elementos con etiqueta **td**), agás na primeira fila, que contén as cabeceiras da táboa (elementos **th**). A idea é, a medida que se percorren as filas, almacenar contidos das celas nun obxecto lista distinto, creando ao final unha estructura de datos "táboa", que será unha lista de listas python. Para mellor manexo desta estructura de datos, finalmente convértese a un **DataFrame** coa axuda da libraría **pandas**, e este será o obxecto que devolva a función.    

In [ ]:
def getBolsa():
    ''' 
    Obtén as cotizacións de bolsa a partires da url de referencia, devolvendo
    os resultados nun obxecto dataframe    
    '''
    
    # url de referencia
    url = "http://www.invertia.com/mercados/bolsa/indices/ibex-35/acciones-ib011ibex35"

    # obtemos unha response mediante a libraría request
    r = requests.get(url)

    # Extraemos texto HTML coa libraría BeautifulSoup
    soup = BeautifulSoup(r.text)
    t = soup.find("table", { "class" : "tb_fichas" })

    # Crease unha táboa personalizada nun obxecto lista    
    myTable = []
    
    # Atopamos as filas da táboa HTML
    filas = t.find_all("tr")
    
    # Percorremos as filas
    for idFila in range(len(filas)):
        
        # Créase unha fila personalizada nun obxecto lista
        myRow =[]
        if idFila == 0: # Cabeceiras
            # Atopamos as cabeceiras da primeira fila
            cabeceiras = filas[idFila].find_all("th")
            # Percorremos as columnas
            for idCabeceira in range(len(cabeceiras)):
                if idCabeceira != 4: # obviamos esta columna sen valores
                    # Eliminanse os retornos de carro do texto das cabeceiras                
                    cabeceiraLimpa = cabeceiras[idCabeceira].text.replace("\n", "")
                    myRow.append(cabeceiraLimpa)
        else:
            # Atopamos as celdas de cada fila
            celas = filas[idFila].find_all("td")
            # Percorremos as columnas
            for idCela in range(len(celas)):
                if idCela != 4: # obviamos esta columna sen valores
                    myRow.append(celas[idCela].text)
        # Engádese a fila á táboa
        myTable.append(myRow)

    # Créase un dataframe a partires da táboa para manexar os datos        
    df = pd.DataFrame(myTable[1:], index=None, columns=myTable[0])
    
    return df

### Función getIdentificadorBursatil(identificador)

Esta función encárgase de realizar a obtención da táboa de datos para posteriormente buscar os correspondentes á cotización do índice solicitado polo usuario, que se lle pasa coma parámetro. Unha vez atopados, preséntaos en liñas consecutivas por pantalla, indicando ademáis, a data e hora da consulta. No caso de non atopar o índice demandado notifícao por pantalla. 

In [ ]:
def getIdentificadorBursatil(identificador):
    ''' Obtén as cotizacións de bolsa dun índice seleccionado '''    
    
    # Obtén as cotizacións de bolsa totais    
    df = getBolsa()
    
    # Comproba se existe o índice
    for indice in range(len(df)): 
        if df.ix[indice]["TKR*"] == identificador:
            '''
            Devolve a data (ano, mes, día,  hora, minuto, segundo) xunto 
            co resultado da consulta
            '''
            print("{0}\n{1} {2}\n{3}\n{4}\n".format("*"*80, \
                  time.strftime("%d/%m/%Y %H:%M:%S"), "*"*60, "*"*80, \
                  df.ix[indice:indice]))           
            '''
            # Outro formato alternativo
            print("{0}".format(df.ix[indice:indice]))
            '''
            return True
            
    print('\nÍndice "{0}" non atopado\n'.format(identificador))
    return False